In [3]:
import requests
import json
import time
import pymongo
from bs4 import BeautifulSoup

In [10]:
# correcting university names
import pandas as pd
university_rankings = pd.read_json('university_rankings.json')

In [13]:
# correcting professor names
university_rankings.index

Index(['Boston University', 'California Institute of Technology',
       'Carnegie Mellon University', 'Colorado School of Mines',
       'Columbia University (Fu Foundation)', 'Cornell University',
       'Duke University (Pratt)', 'Emory University-Georgia Tech University',
       'Georgia Institute of Technology', 'Harvard University',
       'Iowa State University', 'Johns Hopkins University (Whiting)',
       'Louisiana State University--Baton Rouge',
       'Massachusetts Institute of Technology', 'Michigan State University',
       'North Carolina State University',
       'Northwestern University (McCormick)', 'Ohio State University',
       'Oregon State University',
       'Pennsylvania State University--University Park',
       'Princeton University', 'Purdue University--West Lafayette',
       'Rice University (Brown)', 'Stanford University',
       'Texas A&M University--College Station',
       'Texas Tech University (Whitacre)',
       'University of California--Berkeley

In [15]:
# Dictionary of school ids (keys) that map to corrected school name
sids = {124 : ('Boston University', 'Boston University'),
        181:  ('Carnegie Mellon University', 'Carnegie Mellon University'), 
        'NA': ('Colorado School of Mines', 'Colorado School of Mines'),
        278:  ('Columbia University (Fu Foundation)', 'Columbia University'), 
        298:  ('Cornell University', 'Cornell University'),
        1350: ('Duke University (Pratt)', 'Duke University'), 
        340:  ('Emory University-Georgia Tech University', 'Emory University'),
        361:  ('Georgia Institute of Technology', 'Georgia Institute of Technology'), 
        'NA': ('Harvard University', 'Harvard University'),
        452:  ('Iowa State University', 'Iowa State University'), 
        464:  ('Johns Hopkins University (Whiting)','Johns Hopkins University' ),
        3071: ('Louisiana State University--Baton Rouge', 'Louisiana State University'),
        580:  ('Massachusetts Institute of Technology', 'Massachusetts Institute of Technology'), 
        601:  ('Michigan State University', 'Michigan State University'),
        685:  ('North Carolina State University', 'North Carolina State University'),
        709:  ('Northwestern University (McCormick)', 'Northwestern University' ), 
        724:  ('Ohio State University', 'The Ohio State University'),
        742:  ('Oregon State University', 'Oregon State University'),
        758:  ('Pennsylvania State University--University Park', 'Pennsylvania State University'),
        780:  ('Princeton University', 'Princeton University'), 
        783:  ('Purdue University--West Lafayette', 'Purdue University - West Lafayette'),
        799:  ('Rice University (Brown)', 'Rice University'), 
        953:  ('Stanford University', 'Stanford University'),
       1003:  ('Texas A&M University--College Station', 'Texas A&M University at College Station'),
       1011:  ('Texas Tech University (Whitacre)', 'Texas Tech University'),
       1072:  ('University of California--Berkeley', 'University of California Berkeley'), 
       1073:  ('University of California--Davis', 'University of California Davis'),
       1079:  ('University of California--San Diego (Jacobs)', 'University of California San Diego'),
       1077:  ('University of California--Santa Barbara', 'University of California Santa Barbara'),
       1087:  ('University of Colorado--Boulder', 'University of Colorado - Boulder'), 
       1094:  ('University of Delaware', 'University of Delaware'),
       1100:  ('University of Florida', 'University of Florida'), 
       1112:  ('University of Illinois--Urbana-Champaign', 'University of Illinois at Urbana-Champaign'),
       1117:  ('University of Kansas', 'University of Kansas'), 
       1270:  ('University of Maryland--College Park (Clark)', 'University of Maryland'),
       1258:  ('University of Michigan--Ann Arbor', 'University of Michigan'),
       1257:  ('University of Minnesota--Twin Cities', 'University of Minnesota - Twin Cities'),
       1249:  ('University of Nebraska--Lincoln', 'University of Nebraska - Lincoln'), 
       1596:  ('University of Oklahoma', 'University of Oklahoma'),
       1275:  ('University of Pennsylvania', 'University of Pennsylvania'),
       1381:  ('University of Southern California (Viterbi)', 'University of Southern California'),
       1385:  ('University of Tennessee--Knoxville (Tickle)', 'University of Tennessee - Knoxville'),
       1255:  ('University of Texas--Austin (Cockrell)', 'University of Texas at Austin'), 
       3963:  ('University of Tulsa', 'University of Tulsa'),
       1530: ('University of Washington', 'University of Washington'), 
       1256: ('University of Wisconsin--Madison', 'University of Wisconsin - Madison'),
       4282: ('University of Wyoming', 'University of Wyoming'), 
       1349: ('Virginia Tech', 'Virginia Tech'), 
       1222: ('Yale University', 'Yale University')}

In [16]:
with open('sids.json', 'w') as f:
    json.dump(sids, f)

In [10]:
schools

['University of Texas at Austin',
 'Stanford University',
 'Texas A&M University at College Station']

In [26]:
import requests
from bs4 import BeautifulSoup
import json

# school ids for lookup on ratemyprofessors.com
with open('sids.json', 'r') as sids:
    sids = json.load(sids)

# school and faculty names for lookup on ratemyprofessors.com
with open('ut_pge_profs.json', 'r') as f:
    ut_pge_profs = json.load(f)

with open('stanford_pge_profs.json', 'r') as f:
    stanford_pge_profs = json.load(f)

with open('tamu_pge_profs.json', 'r') as f:
    tamu_pge_profs = json.load(f)

# university rankings and other info pulled from usnews website
with open('university_rankings.json', 'r') as f:
    university_rankings = json.load(f)

headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/68.0.3440.106 Safari/537.36'}

def rating_and_tags(sid, school_name, faculty_name):
    '''Gives rating and tags for a professor pulled from ratemyprofessors.com
       given sid (school's id as per ratemyprofessors.com), school_name and faculty_name
    '''

     # Getting rating url by searching school and faculty name
    url = f"http://www.ratemyprofessors.com/search.jsp?queryoption=HEADER&queryBy=teacherName&schoolName={school_name}&schoolID={sid}&query={faculty_name}"
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')

    try:
        rating_url = "http://www.ratemyprofessors.com" + soup.select("li.listing.PROFESSOR")[0].find("a")["href"]

        # Getting rating and profile tags for the rating url
        response2 = requests.get(rating_url, headers=headers)
        soup2 = BeautifulSoup(response2.content, 'html.parser')
        rating = float(soup2.select("div.grade")[0].get_text())
        tags = [tag_desc.get_text() for tag_desc in soup2.select("span.tag-box-choosetags")]

    except:
        rating, tags = 'N/A', 'N/A'

    return {'rating': rating, 'tags': tags}

In [27]:
# Creating complete university database
university = university_rankings.copy()
schools_for_lookup = ['University of Texas at Austin', 'Stanford University', 'Texas A&M University at College Station']
sids_for_lookup = ['1255', '953', '1003']

for sid, school in  zip(sids_for_lookup, schools_for_lookup):
  
    if school == 'University of Texas at Austin':
        university['petroleum-engineering']['University of Texas--Austin (Cockrell)'] ['faculty_names'] = {}
        for faculty in ut_pge_profs['faculty_names']:
            university['petroleum-engineering']['University of Texas--Austin (Cockrell)'] ['faculty_names'][faculty] = rating_and_tags(sid, school, faculty)
    
    if school == 'Stanford University':
        university['petroleum-engineering']['Stanford University'] ['faculty_names'] = {}
        for faculty in stanford_pge_profs['faculty_names']:
            university['petroleum-engineering']['Stanford University'] ['faculty_names'][faculty] = rating_and_tags(sid, school, faculty)
    
    if school == 'Texas A&M University at College Station':
        university['petroleum-engineering']['Texas A&M University--College Station'] ['faculty_names'] = {}
        for faculty in tamu_pge_profs['faculty_names']:
            university['petroleum-engineering']['Texas A&M University--College Station'] ['faculty_names'][faculty] = rating_and_tags(sid, school, faculty)

In [28]:
university['petroleum-engineering']['Texas A&M University--College Station'] 

{'location': 'College Station, TX',
 'score': 4.3,
 'rank': 2,
 'faculty_names': {'Akhil Datta-Gupta': {'rating': 5.0,
   'tags': [' LOTS OF HOMEWORK (1)',
    " Skip class? You won't pass. (1)",
    ' Amazing lectures (1)']},
  'A. Daniel Hill': {'rating': 4.0, 'tags': []},
  'Nayef Alyafei': {'rating': 'N/A', 'tags': 'N/A'},
  'Albertus Retnanto': {'rating': 'N/A', 'tags': 'N/A'},
  'Berna Hascakir': {'rating': 'N/A', 'tags': 'N/A'},
  'W. David Kennedy': {'rating': 'N/A', 'tags': 'N/A'},
  'Duane McVay': {'rating': 3.0, 'tags': []},
  'David Schechter': {'rating': 3.0,
   'tags': [' Respected (1)', ' Tough Grader (1)', ' Get ready to read (1)']},
  'Ding Zhu': {'rating': 'N/A', 'tags': 'N/A'},
  'Eduardo Gildin': {'rating': 'N/A', 'tags': 'N/A'},
  'Mohamed Fadlelmula': {'rating': 'N/A', 'tags': 'N/A'},
  'Fred Dupriest': {'rating': 'N/A', 'tags': 'N/A'},
  'George Moridis': {'rating': 'N/A', 'tags': 'N/A'},
  'Dominique Guerillot': {'rating': 'N/A', 'tags': 'N/A'},
  'Hadi Nasrabad

In [29]:


# Writing results to a json file
with open('database.json', 'w') as f:
    json.dump(university, f)